## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
#data.head(30)
data.info() #файл прочитан и выведена инфа о нем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Как видно из данных, некоторые значения в столбцах "days_employed" и "total_income" пропущены, что говорит о каких-либо ошибок при занесении или передачи данных. Также прослеживаются отрицательные значения в столбце days_employed, что также противоречит здравому смыслу

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
import numpy as np
w_avr = {}
# создаю словарь с медианными значениями зарплат для каждой категории

for i in data.income_type.value_counts().index:
    sotrudnik_avr = data.loc[(data['income_type']== i), :].total_income.mean()
    w_avr[i] = sotrudnik_avr
    
# с помощью функции и метода aplly заменяю пустые значения на медианные
def fill_income(row):
    key = row['income_type']
    if pd.isnull(row['total_income']):
        return w_avr[key]
    else:
        return row['total_income']
data['total_income'] = data.apply(fill_income, axis=1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.days_employed = data.days_employed.apply(abs)
days_avr = {}
# создаю словарь с медианными значениями стажа для каждой категории

for i in data.income_type.value_counts().index:
    days_med = data.loc[(data['income_type']== i), :].days_employed.mean()
    days_avr[i] = days_med
    
# с помощью функции и метода aplly заменяю пустые значения на медианные
def fill_days(row):
    key = row['income_type']
    if pd.isnull(row['days_employed']):
        return days_avr[key]
    else:
        return row['days_employed']
    
data['days_employed'] = data.apply(fill_days, axis=1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">Я теперь создал словари со средними значениями по каждой категории по столбцам 'total_income' и 'days_employed' и заполнил данные с помощью этих значений. Конечно, данных подход немного занижает средние цифры (ведь учитывались и пропущенные строки), но этим можно пренебречь ради времени и эффективности.</div>


### Вывод

<div class="alert alert-block alert-info">Были обнаружены пустые значения в столбцах total_income и days_employed. Я взял за основу столбец 'income_type', и далее нашел медианные значения каждой категории для столюцов 'total_income' and 'days_employed'. Данной подход поможет нам заполнить пустые значения наиболее оптимальными средними значениями для именно той категории, где пропущеннно значение. Так мы сможем восстановить данные наиболее корректно. </div>

### Замена типа данных

In [5]:
data.days_employed = data.days_employed.round() # округляем стаж до целых жней
data.total_income = data.total_income.round() # округляем доход до целых единиц, так как копейки - очень 
#несущественная величина в данном исследовании
data.total_income = data.total_income.astype('int')
data.days_employed = data.days_employed.astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

<div class="alert alert-block alert-info">Сначала округлил значения days_employed и total_income и далее привел их к целому формату числа. В случае с total_income это, мне кажется, логично, ведь сотые доли рубля (да даже доллара) не играют важную роль в данном вопросе(исследование кредитоспособности, учитывая инфляцию рубля)</div>

Теперь, опять же, надо заменить тип float на int. Так как тип данных float используется в столбцах total_income и days_employed, то изменим их тип. Также заметим, что в столбце days_employed есть отрицательные значения, скорее всего перепутались знаки. Поэтому применим функцию модуля к ним (сделал в предыдущем пункте).

### Обработка дубликатов

In [6]:
data['education'] = data['education'].str.lower() #приводим значения столбца к единому регистру
data['family_status'] = data['family_status'].str.lower()

def correct_education(value):
    if value in ['высшее', "ученая степень"] :
        return "есть высшее образование"
    return "нет высшего образования"
    
data.education = data.education.apply(correct_education)

data.duplicated().sum() #есть 71 дупликатов, удалим строки - дубликаты
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum() # теперь нет дупликатов

0

### Вывод

<div class="alert alert-block alert-info">В данном датасете я решил категоризировать образование на 'есть высшее образование' и "нет высшего образования", так как, по-моему мнению, в условиях выдачи кредита именно эти две категории играют ключевые роли. А какая именно степень у тебя - это вторичные категории и могут успользоваться только для более глубокого ML при написании алгоритма определения кредитоспособности.</div>
<div><font size="3" color="orange" face="Times New Roman">До этого я привел все к единому регистру и далее посмотрел на кол-во дупликатов. И обнаружилось 71 дупликатов, которые я успешно удалил</font></div>

### Лемматизация

In [7]:
from pymystem3 import Mystem
from collections import Counter
all_purposes = []
all_purposes_2 = []
m = Mystem()
lemmas = data['purpose'].apply(m.lemmatize)
for i in lemmas:
    for b in i:
        all_purposes.append(b) # закинул все лемматизированные слова в один список
for el in all_purposes:
    if el !='покупка' and el != 'приобретение' and el not in ['операция', '\n', ' ']: # убираю все бесполезные слова, которые не передают суть кредита
        all_purposes_2.append(el)
print(Counter(all_purposes_2)) # считаю кол-во каждого лемматизированного слова 

Counter({'недвижимость': 6351, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'профильный': 436})


### Вывод

Как видно из анализа целей кредита, большинству людей нужные кредиты для следующих целей: жилье, автомобиль, образование, семья. Эти категории мы и выделим в след пункте

### Категоризация данных

In [8]:
def make_target(val):
    var = m.lemmatize(val)
    if 'недвижимость'in var or "жилье" in var or'строительство' in var or 'жилой' in var or 'ремонт' in var:
        return 'жилье'
    elif "автомобиль" in var or "поддержанный" in var:
        return "автомобиль"
    elif "образование" in var:
        return "образование"
    elif "семья" in var or 'свадьба' in var:
        return "семья"

data['purp_category'] = data['purpose'].apply(make_target)
data.info() # Добавил категории по целям, и с помощью функции и метода apply применил эту функцию к столбццу purpose
# нет пропущенных значений, значит все успешно

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
purp_category       21454 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

Ну тут я просто уже применил выделенные категории к датасету.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
data.children.value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [10]:
def correct_children(privet):
    if privet < 0:
        return -privet
    elif privet == 20:
        return 2
    else:
        return privet

data.children = data.children.apply(correct_children)
data.children.value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

<div class="alert alert-block alert-info">Как видно, были пару артефактов. Во-первых, отрицательные значения. Можно предположить, что данная ошибка была вызвана случайной постановкой знака минус, поэтому я просто взял модуль данного значения. Также в некоторых случаях было значения 20, что также противоречит здравому смыслу (у нас же не ОАЭ). Поэтому, я предположу, что была случайная постановка знака 0 в конце и я искоренил данную проблему, убрав 0</div>

In [11]:
# посчитаем эту зависимость так: будем смотреть отношения тех, у кого есть долги к общему количеству людей в определенной категории
# эти категории считаем так: 0 детей, 1-2 детей и 3+ детей
freechild= len(data.loc[data['children']==0, :].index)
in_debt_freechild = len(data.loc[(data['children']==0) & data.debt == 1, :].index)
few_child= len(data.loc[(data['children']<3) & (data['children']>0), :].index)
in_debt_few_child = len(data.loc[((data['children']<3) & (data['children']>0)) & data.debt == 1, :].index)
many_child= len(data.loc[data['children']>2, :].index)
in_debt_many_child = len(data.loc[(data['children']>2) & data.debt == 1, :].index)
print("Процент должников среди тех,у которых нет детей", "{:.2f}".format(in_debt_freechild/freechild))
print("Процент должников среди тех, у которых 1-2 детей", "{:.2f}".format(in_debt_few_child/few_child))
print("Процент должников среди тех, у которых больше 3 детей", "{:.2f}".format(in_debt_many_child/many_child))

Процент должников среди тех,у которых нет детей 0.08
Процент должников среди тех, у которых 1-2 детей 0.09
Процент должников среди тех, у которых больше 3 детей 0.08


In [12]:
data.groupby('debt').children.mean()

debt
0    0.475016
1    0.543366
Name: children, dtype: float64

### Вывод

Как можно наблюдать, есть зависиомсть между непогашением долга в срок и кол-вом детей нет. А именно, можно сказать, что те, у кого нет детей, чуть более кредитоплатежны, чем те, у кого они есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
data_2 = data.groupby('family_status').debt.value_counts() #мультииндексная таблица, отражающая данные о семейном статусе и кол-ве должников

In [14]:
data_2

family_status          debt
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3763
                       1         388
женат / замужем        0       11408
                       1         931
не женат / не замужем  0        2536
                       1         274
Name: debt, dtype: int64

In [15]:
percent_of_debetors= [round(data_2[i]/(data_2[i]+data_2[i-1]), 3) for i in range(1,10,2)]
table_of_debetors= pd.DataFrame({'в разводе': [percent_of_debetors[0]], 'вдовец / вдова':[percent_of_debetors[1]], 'гражданский брак': [percent_of_debetors[2]], 'женат / замужем': [percent_of_debetors[3]],'не женат / не замужем': [percent_of_debetors[4]]})
table_of_debetors

,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
0,0.071,0.066,0.093,0.075,0.098


### Вывод

<div class="alert alert-block alert-info">Как видно из преведенный выше таблице, самый высокий процент должников у холостяков. Самый низкий - это у тех, кто потерял своего супруга. Возможно, холостяки более беспечны к обязательствам, именно поэтому не так внимательно следят за пунктуальностью платежей по кредиту. 
Также можно заметить, что люди в браке более внимательны к кредиту, нежели люди в гражданском браке. Это может быть связано с тем, что люди в легальном браке помогают друг другу закрывать кредиты в срок, в то время как люди в гражданском браке, возмлжно, меньше взаимопомогают друг другу (возможно, просто проживают вместе, без взаимной ответственности).
Низник процент в людей, супруг которых умер, можно объяснить тем, что эти люди пережили много событий, в том числе трагических, и они более ответственны за свою жизнь.</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
data.groupby('debt').total_income.mean()

debt
0    167806.617359
1    163190.697300
Name: total_income, dtype: float64

In [17]:
# Разделим людей на категории: низнший класс- менее 30000 руб/мес, средний класс - от 30 000 до 60 000
#состоятельный средний - от 60 до 100 тыс руб, и богатые от 100 000 руб
#делал это на основе данных https://www.drive2.ru/b/2393127/ , немного подкорректировал под собственные взгляды
# и в каждом классе посмотрим на отношение должников к общему кол-ву людей в классе (в процентах)
low_income = data.loc[data['total_income']<30000, :].count()[0]
debt_low_income = data.loc[(data['total_income']<30000) & (data['debt'] == 1), :].count()[0]

medium_income = data.loc[(data['total_income']>=30000)&(data['total_income']<=60000), :].count()[0]
debt_medium_income = data.loc[((data['total_income']>=30000)&(data['total_income']<=60000)) & (data['debt'] == 1), :].count()[0]

high_income = data.loc[(data['total_income']>60000)&(data['total_income']<=100000), :].count()[0]
debt_high_income = data.loc[((data['total_income']>60000)&(data['total_income']<=100000)) & (data['debt'] == 1), :].count()[0]

very_high_income = data.loc[data['total_income']>100000, :].count()[0]
very_high_debt_income = data.loc[(data['total_income']>100000) & (data['debt'] == 1), :].count()[0]
print('Процент должников у низшего класса равен {:.2%}'.format(debt_low_income/low_income))
print('Процент должников у среднего класса равен {:.2%}'.format(debt_medium_income/medium_income))
print('Процент должников у среднего состоятельного равен {:.2%}'.format(debt_high_income/high_income))
print('Процент должников у богатого класса равен {:.2%}'.format(very_high_debt_income/very_high_income))

Процент должников у низшего класса равен 9.09%
Процент должников у среднего класса равен 5.99%
Процент должников у среднего состоятельного равен 8.34%
Процент должников у богатого класса равен 8.16%


### Вывод

<div class="alert alert-block alert-info">Как видно из вышеперечисленных данных, можно сказать следующее. Так, однозначно можно сказать, что бедный класс (менее 30 тыс в мес) - самые не кредитоплатежный. Средний класс (от 30 до 60 тыс руб в мес) - самый отвественный в плане погашения кредита. Далее - идет более состоятельный слой населения, и там процент должников подскакивает. Думаю, это связанно с тем, что данные классы, имея существенные (по меркам РФ) доходы, взяли на себя кредиты, переоценив свою платежеспособность. Но опять же, платежеспособность - это сочетания фин грамотности и ответственности. И эти способности не коррелируют с уровнем дохода, именно поэтому, на мой взгляд, не правильно ставить какую-либо зависимость между уровнем дохода и платежеспоссобностью (Данное утверждение применимо лишь к среднему и высшим классам)</div>

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
data_3 = data.groupby('purp_category').debt.value_counts()
data_3

purp_category  debt
автомобиль     0        3903
               1         403
жилье          0       10029
               1         782
образование    0        3643
               1         370
семья          0        2138
               1         186
Name: debt, dtype: int64

In [19]:
percent_of_debetors_by_purp= [round(100*data_3[i]/(data_3[i]+data_3[i-1]), 3) for i in range(1,8,2)]
table_of_debetors_by_purp = pd.Series({'автомобиль': percent_of_debetors_by_purp[0], 'жилье':percent_of_debetors_by_purp[1],'образование':percent_of_debetors_by_purp[2],'семья':percent_of_debetors_by_purp[3]})
table_of_debetors_by_purp

автомобиль     9.359
жилье          7.233
образование    9.220
семья          8.003
dtype: float64

### Вывод

<div class="alert alert-block alert-info">Наибольший невозврат кредитов есть в категориях "образования" и "автомобиль", в то время как процент должников в таких категориях, как "жилье" и "семья" примерно одинаков
Возможно, это связнно с тем, что кредиты на Образование и Машины более рискованные, так как 1)На кредит берут обычно статусную машину, которую человек не может себе позволить. И эта статусность зачастую не подтверждается фактически, отсюда и проблемы с долгами
    2) Кредит на образование - это риск, где ты либо можешь стать востребованным специалистом, либо не сможешь достаточно хорошо обучиться (лень, непосильная программа и тд) что и создает тебе преграду для погашения кредита</div>
    
<div class="alert alert-block alert-info"> А кредиты на недвижимость и семью имеет под собой более фундаментальные причины, ведь жилье - это главный актив у человека, и поэтому человек будет стараться изо всех сил удержать его. Что касается семьи, кредит на создание семьи обеспечен стараниями двоих супругов, что также делает его болеенадежным для банка.</div>

### Шаг 4. Общий вывод

<div class="alert alert-block alert-info">В итоге можно сказать, что во время изучения данных были сделаны следующие выводы:

1)Самый большой процент невозврата у тех людей, кто еще не был в браке (каждый десятый). Причем люди, которые были в браке, более отвественные к погашению кредита, по сравненис с теми, кто в браке не был.

2) Люди с детьми более склонны к невыполнению обязательств по кредиту, нежели те, у кого детей нет

3) Люди с маленьким доходом (менее 30 тыс руб) - самый неплатежеспособной сегмент. Но далее четкой корреляции не прослеживается. Скорее всего, тут более главную роль играет самодисциплина и умение грамотно распоряжаться деньгами

4)наиболее надежные кредиты - это кредиты, связанные с жильем и кредиты, направленные на создание семьи. Так как они направленны на удовлетворение важнейших биологических и социальных потребностей, что и обуславливает надежность погашения. Самый рисковый кредит - это кредит на покупку машин и образование, так как природа причин взятия этих кредитов рискованная.</div>



### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.